In [3]:
import os, json, re, statistics
from openai import OpenAI

In [ ]:
DATA = [
  {
    "task_id":"T01",
    "text":"明天下午3点在A楼402和张三、李四开评审会，讨论界面原型。",
    "gold":{"task_id":"T01","date":"2026-01-28","time":"15:00","participants":["张三","李四"],"location":"A楼402","action":"开评审会，讨论界面原型"}
  },
  {
    "task_id":"T02",
    "text":"请于1月30日 09:30 到图书馆二楼会议室，王敏和陈浩一起做开题沟通。",
    "gold":{"task_id":"T02","date":"2026-01-30","time":"09:30","participants":["王敏","陈浩"],"location":"图书馆二楼会议室","action":"做开题沟通"}
  },
  {
    "task_id":"T03",
    "text":"周五中午12:00，和刘老师在食堂门口碰头，随后去实验室看设备。",
    "gold":{"task_id":"T03","date":"2026-01-30","time":"12:00","participants":["刘老师"],"location":"食堂门口","action":"碰头后去实验室看设备"}
  },
  {
    "task_id":"T04",
    "text":"后天10点，地点B座1层大厅，参加人：小王/小赵。事项：布置展板。",
    "gold":{"task_id":"T04","date":"2026-01-29","time":"10:00","participants":["小王","小赵"],"location":"B座1层大厅","action":"布置展板"}
  },
  {
    "task_id":"T05",
    "text":"1/31 16:20 在C楼-305 找周工确认需求文档。",
    "gold":{"task_id":"T05","date":"2026-01-31","time":"16:20","participants":["周工"],"location":"C楼-305","action":"确认需求文档"}
  },
  {
    "task_id":"T06",
    "text":"今天晚上7点到7点半，线上腾讯会议，参会：Alice、Bob，内容：同步进度。",
    "gold":{"task_id":"T06","date":"2026-01-27","time":"19:00","participants":["Alice","Bob"],"location":"腾讯会议(线上)","action":"同步进度"}
  },
  {
    "task_id":"T07",
    "text":"明天09:05在主楼101和小李做论文修改确认。",
    "gold":{"task_id":"T07","date":"2026-01-28","time":"09:05","participants":["小李"],"location":"主楼101","action":"论文修改确认"}
  },
  {
    "task_id":"T08",
    "text":"后天下午2点半，地点：E楼210，和赵老师讨论实验方案。",
    "gold":{"task_id":"T08","date":"2026-01-29","time":"14:30","participants":["赵老师"],"location":"E楼210","action":"讨论实验方案"}
  },
  {
    "task_id":"T09",
    "text":"本周五15:15在行政楼3层会议室，参会：王强、孙悦。事项：项目立项汇报。",
    "gold":{"task_id":"T09","date":"2026-01-30","time":"15:15","participants":["王强","孙悦"],"location":"行政楼3层会议室","action":"项目立项汇报"}
  },
  {
    "task_id":"T10",
    "text":"周六上午10:30在体育馆北门集合，和小陈、小何参加羽毛球训练。",
    "gold":{"task_id":"T10","date":"2026-01-31","time":"10:30","participants":["小陈","小何"],"location":"体育馆北门","action":"参加羽毛球训练"}
  },
  {
    "task_id":"T11",
    "text":"1月28日晚上20:00 Zoom线上会议，参会：Dana, Evan。议题：需求评审。",
    "gold":{"task_id":"T11","date":"2026-01-28","time":"20:00","participants":["Dana","Evan"],"location":"Zoom(线上)","action":"需求评审"}
  },
  {
    "task_id":"T12",
    "text":"明天中午12点在食堂二楼，和周工、许工对齐接口方案。",
    "gold":{"task_id":"T12","date":"2026-01-28","time":"12:00","participants":["周工","许工"],"location":"食堂二楼","action":"对齐接口方案"}
  },
  {
    "task_id":"T13",
    "text":"后天08:00到实验室A区，和黄同学做数据标注。",
    "gold":{"task_id":"T13","date":"2026-01-29","time":"08:00","participants":["黄同学"],"location":"实验室A区","action":"做数据标注"}
  },
  {
    "task_id":"T14",
    "text":"本周五18:40在地铁口A出口见面，参与者：小林。目的：取快递。",
    "gold":{"task_id":"T14","date":"2026-01-30","time":"18:40","participants":["小林"],"location":"地铁口A出口","action":"取快递"}
  },
  {
    "task_id":"T15",
    "text":"下周一 09:00 在科研楼B402，参会：李博、张宁，内容：周例会。",
    "gold":{"task_id":"T15","date":"2026-02-02","time":"09:00","participants":["李博","张宁"],"location":"科研楼B402","action":"周例会"}
  },
  {
    "task_id":"T16",
    "text":"下周二14:00在南门咖啡店和杨老师沟通投稿计划。",
    "gold":{"task_id":"T16","date":"2026-02-03","time":"14:00","participants":["杨老师"],"location":"南门咖啡店","action":"沟通投稿计划"}
  },
  {
    "task_id":"T17",
    "text":"2月1日 11:10 到博物馆入口，和小吴、小钱做展陈拍摄。",
    "gold":{"task_id":"T17","date":"2026-02-01","time":"11:10","participants":["小吴","小钱"],"location":"博物馆入口","action":"做展陈拍摄"}
  },
  {
    "task_id":"T18",
    "text":"周日晚上21:00在微信群语音，参会：Tom、Jerry，事项：确认分工。",
    "gold":{"task_id":"T18","date":"2026-02-01","time":"21:00","participants":["Tom","Jerry"],"location":"微信群语音(线上)","action":"确认分工"}
  },
  {
    "task_id":"T19",
    "text":"1月29日16:00到D楼105，找方老师签字。",
    "gold":{"task_id":"T19","date":"2026-01-29","time":"16:00","participants":["方老师"],"location":"D楼105","action":"签字"}
  },
  {
    "task_id":"T20",
    "text":"1月30日 13:20 在公司前台，参与者：陈经理。事项：领取门禁卡。",
    "gold":{"task_id":"T20","date":"2026-01-30","time":"13:20","participants":["陈经理"],"location":"公司前台","action":"领取门禁卡"}
  },
]


In [5]:
TODAY = "2026-01-27"

In [6]:
ROLE = "你是一个严格的信息抽取助手。"
CONTEXT = (
  "请从用户文本中抽取日程信息，输出JSON，字段含义：\n"
  "- date: YYYY-MM-DD\n- time: HH:MM(24小时)\n- participants: 人名列表\n- location: 地点(线上则写'平台(线上)')\n- action: 事项/目的(尽量短)\n"
  f"已知今天日期是 {TODAY}，'明天/后天/周五' 等需换算成具体日期。\n"
)
CONSTRAINT = (
  "硬性要求：\n"
  "1) 只能输出一个JSON对象，JSON之外不得出现任何文字。\n"
  "2) JSON只能包含这6个key：task_id,date,time,participants,location,action（不得多也不得少）。\n"
  "3) participants必须是数组。\n"
)
FEWSHOT = [
  ("用户：明天9点在主楼101和小李讨论论文。", 
   {"task_id":"EX","date":"2026-01-28","time":"09:00","participants":["小李"],"location":"主楼101","action":"讨论论文"})
]

In [7]:
def build_messages(variant_name: str, item):
  parts = []
  if variant_name != "-ROLE":
    parts.append(ROLE)
  if variant_name != "-CONTEXT":
    parts.append(CONTEXT)
  if variant_name != "-CONSTRAINT":
    parts.append(CONSTRAINT)

  system_prompt = "\n".join([p for p in parts if p])

  messages = [{"role":"system","content":system_prompt}]
  if variant_name != "-FEWSHOT":
    for ex_text, ex_json in FEWSHOT:
      messages.append({"role":"user","content":ex_text})
      messages.append({"role":"assistant","content":json.dumps(ex_json, ensure_ascii=False)})

  messages.append({"role":"user","content":f"task_id={item['task_id']}\n文本：{item['text']}"})
  return messages

# ========= 3) 自动打分（≤3指标）=========
def norm(s: str) -> str:
  if s is None: return ""
  s = s.strip()
  s = re.sub(r"\s+", "", s)
  s = s.replace("，", ",").replace("。", "").replace("：", ":")
  return s

ALLOWED_KEYS = {"task_id","date","time","participants","location","action"}

def score_one(output_text: str, gold: dict):
  # Compliance
  try:
    obj = json.loads(output_text)
  except Exception:
    return {"compliance":0, "slot_acc":0.0}

  if set(obj.keys()) != ALLOWED_KEYS:
    return {"compliance":0, "slot_acc":0.0}
  if not isinstance(obj.get("participants"), list):
    return {"compliance":0, "slot_acc":0.0}

  # Slot accuracy (5 slots)
  acc = 0
  acc += int(norm(obj.get("date")) == norm(gold["date"]))
  acc += int(norm(obj.get("time")) == norm(gold["time"]))
  acc += int(norm(obj.get("location")) == norm(gold["location"]))
  acc += int(norm(obj.get("action")) == norm(gold["action"]))
  acc += int(set(map(norm, obj.get("participants", []))) == set(map(norm, gold["participants"])))
  return {"compliance":1, "slot_acc":acc/5.0}

In [ ]:
# ========= 4) 调用 DeepSeek + 跑对比 ========
# DeepSeek: base_url=https://api.deepseek.com, model=deepseek-chat (OpenAI-compatible)  :contentReference[oaicite:5]{index=5}
client = OpenAI(
  api_key="sk-0b94eb5945114830b12de5509a78ae5d",
  base_url="https://api.deepseek.com"
)

VARIANTS = ["FULL", "-ROLE", "-CONTEXT", "-CONSTRAINT", "-FEWSHOT"]

def run():
  print("variant\tcompliance\tslot_acc\tavg_tokens")
  for v in VARIANTS:
    comp_list, acc_list, tok_list = [], [], []
    for item in DATA:
      messages = build_messages(v, item)
      resp = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        temperature=0,
        stream=False
      )
      text = resp.choices[0].message.content
      sc = score_one(text, item["gold"])
      comp_list.append(sc["compliance"])
      acc_list.append(sc["slot_acc"])
      tok_list.append(getattr(resp, "usage", {}).total_tokens if hasattr(resp, "usage") and resp.usage else 0)

    print(
      f"{v}\t"
      f"{sum(comp_list)/len(comp_list):.2f}\t\t"
      f"{sum(acc_list)/len(acc_list):.2f}\t\t"
      f"{statistics.mean(tok_list):.1f}"
    )

run()

variant	compliance	slot_acc	avg_tokens
FULL	1.00		0.93		309.8
-ROLE	1.00		0.90		302.2
-CONTEXT	1.00		0.87		216.3
-CONSTRAINT	1.00		0.93		258.7
-FEWSHOT	1.00		0.93		242.3


In [9]:
for i in range(5):
    print(f"round {i+1}")
    run()

round0
variant	compliance	slot_acc	avg_tokens
FULL	1.00		0.93		309.8
-ROLE	1.00		0.90		302.2
-CONTEXT	1.00		0.87		216.3
-CONSTRAINT	1.00		0.93		258.7
-FEWSHOT	1.00		0.90		243.8
round1
variant	compliance	slot_acc	avg_tokens
FULL	1.00		0.93		309.8
-ROLE	1.00		0.90		302.2
-CONTEXT	1.00		0.87		216.3
-CONSTRAINT	1.00		0.93		258.7
-FEWSHOT	1.00		0.93		242.3
round2
variant	compliance	slot_acc	avg_tokens
FULL	1.00		0.93		309.8


KeyboardInterrupt: 